In [18]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is 50C4-BF99

 Directory of c:\Users\gerba\Programming\Flatiron School Data Science Bootcamp - April Cohort 2023\capstone\data_processing_files

07/12/2023  01:02 PM    <DIR>          .
07/11/2023  04:44 PM    <DIR>          ..
07/11/2023  04:28 PM            32,440 email_preprocessing.ipynb
07/11/2023  04:28 PM            32,417 email_preprocessing_v2.ipynb
07/12/2023  01:23 PM             5,264 email_preprocessing_v3.ipynb
               3 File(s)         70,121 bytes
               2 Dir(s)  1,105,631,662,080 bytes free


In [19]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [21]:
# import data
data = pd.read_csv('../data/combined_data/combined_data.csv')

In [23]:
data.head().T

,0,1,2,3,4
Index,1,2,3,4,5
Message Body,Message-ID: <15886558.1075840010371.JavaMail.e...,Message-ID: <15680772.1075840010630.JavaMail.e...,Message-ID: <19004769.1075840009908.JavaMail.e...,Message-ID: <20348456.1075839979614.JavaMail.e...,Message-ID: <9744175.1075839979640.JavaMail.ev...
Number of Words,519,224,150,187,187
Number of Stop Words,185,77,23,20,20
Number of Unique Words,312,170,140,107,107
Ratio of Lowercase to Uppercase,0.722543,0.754464,0.406667,0.374332,0.374332
Number of Exclamation Points,0,0,0,0,0
Number of Unique Stemmed Words,233,134,132,110,110
Number of Lemmatized Words,240,136,133,112,112
Cleaned Body,message-id: <15886558.1075840010371.javamail.e...,message-id: <15680772.1075840010630.javamail.e...,message-id: <19004769.1075840009908.javamail.e...,message-id: <20348456.1075839979614.javamail.e...,message-id: <9744175.1075839979640.javamail.ev...


In [24]:
import pandas as pd

def drop_columns(df, columns):
  """Drops all columns except the ones specified.

  Args:
    df: The Pandas DataFrame.
    columns: The list of columns to keep.

  Returns:
    A Pandas DataFrame with the specified columns.
  """

  return df[columns]

if __name__ == '__main__':
  df = data
  columns = ['Cleaned Body', 'Target']

  df = drop_columns(df, columns)

  print(df.head())


                                        Cleaned Body  Target
0  message-id: <15886558.1075840010371.javamail.e...       0
1  message-id: <15680772.1075840010630.javamail.e...       0
2  message-id: <19004769.1075840009908.javamail.e...       0
3  message-id: <20348456.1075839979614.javamail.e...       0
4  message-id: <9744175.1075839979640.javamail.ev...       0


In [25]:
df.head().T

,0,1,2,3,4
Cleaned Body,message-id: <15886558.1075840010371.javamail.e...,message-id: <15680772.1075840010630.javamail.e...,message-id: <19004769.1075840009908.javamail.e...,message-id: <20348456.1075839979614.javamail.e...,message-id: <9744175.1075839979640.javamail.ev...
Target,0,0,0,0,0


In [32]:
# export df as csv
df.to_csv('../data/combined_data/combined_data_cleaned.csv', index=True)

In [33]:
import re

def extract_features(email):
  """Extracts features from an email message.

  Args:
    email: The email message as a string.

  Returns:
    A dictionary of features.
  """

  features = {}

  # Number of words in the body.
  body_words = re.findall(r"\w+", email.body)
  features["body_words"] = len(body_words)

  # Number of characters in the body.
  body_characters = len(email.body)
  features["body_characters"] = body_characters

  # Number of words in the subject.
  subject_words = re.findall(r"\w+", email.subject)
  features["subject_words"] = len(subject_words)

  # Number of characters in the subject.
  subject_characters = len(email.subject)
  features["subject_characters"] = subject_characters

  # Body richness.
  features["body_richness"] = features["body_words"] / features["body_characters"]

  # Subject richness.
  features["subject_richness"] = features["subject_words"] / features["subject_characters"]

  # Number of links in the email.
  links = re.findall(r"(http|https)://\S+", email.body)
  features["url_count"] = len(links)

  # Number of external links in the email.
  external_links = [link for link in links if not link.startswith("http://localhost")]
  features["url_external_count"] = len(external_links)

  # Number of domains in the email.
  domains = set([link.split("/")[2] for link in links])
  features["url_domain_count"] = len(domains)

  # Whether the body contains HTML.
  if re.search(r"<html>", email.body):
    features["body_html"] = 1
  else:
    features["body_html"] = 0

  # Maximum number of periods in a URL.
  max_periods = 0
  for link in links:
    num_periods = link.count(".")
    if num_periods > max_periods:
      max_periods = num_periods
  features["url_max_periods"] = max_periods

  return features


In [34]:
import pandas as pd
import re

def extract_features(df):
  """Extracts features from a Pandas DataFrame.

  Args:
    df: The Pandas DataFrame.

  Returns:
    A Pandas DataFrame with the added features.
  """

  new_columns = ['body_words', 'body_characters', 'subject_words', 'subject_characters',
                 'body_richness', 'subject_richness', 'url_count', 'url_external_count',
                 'url_domain_count', 'body_html', 'url_max_periods']

  for column in new_columns:
    df[column] = None

  for index, row in df.iterrows():
    email = row['Cleaned Body']
    features = extract_features(email)

    for column, value in features.items():
      df.loc[index, column] = value

  return df[new_columns]

if __name__ == '__main__':
  df = pd.read_csv('../data/combined_data/combined_data_cleaned.csv')
  df = extract_features(df)

  print(df.head())


TypeError: 'str' object does not support item assignment

In [35]:
import pandas as pd
import re

def extract_features(email):
  """Extracts features from an email message.

  Args:
    email: The email message as a string.

  Returns:
    A dictionary of features.
  """

  features = {}

  # Number of words in the body.
  body_words = re.findall(r"\w+", email.body)
  features["body_words"] = len(body_words)

  # Number of characters in the body.
  body_characters = len(email.body)
  features["body_characters"] = body_characters

  # Number of words in the subject.
  subject_words = re.findall(r"\w+", email.subject)
  features["subject_words"] = len(subject_words)

  # Number of characters in the subject.
  subject_characters = len(email.subject)
  features["subject_characters"] = subject_characters

  # Body richness.
  features["body_richness"] = features["body_words"] / features["body_characters"]

  # Subject richness.
  features["subject_richness"] = features["subject_words"] / features["subject_characters"]

  # Number of links in the email.
  links = re.findall(r"(http|https)://\S+", email.body)
  features["url_count"] = len(links)

  # Number of external links in the email.
  external_links = [link for link in links if not link.startswith("http://localhost")]
  features["url_external_count"] = len(external_links)

  # Number of domains in the email.
  domains = set([link.split("/")[2] for link in links])
  features["url_domain_count"] = len(domains)

  # Whether the body contains HTML.
  if re.search(r"<html>", email.body):
    features["body_html"] = 1
  else:
    features["body_html"] = 0

  # Maximum number of periods in a URL.
  max_periods = 0
  for link in links:
    num_periods = link.count(".")
    if num_periods > max_periods:
      max_periods = num_periods
  features["url_max_periods"] = max_periods

  return features

def main():
  df = pd.read_csv('../data/combined_data/combined_data_cleaned.csv')

  # Create a new DataFrame with the features.
  new_df = pd.DataFrame()
  for index, row in df.iterrows():
    email = row['Cleaned Body']
    features = extract_features(email)
    new_df = new_df.append(features, ignore_index=True)

  print(new_df.head())

if __name__ == '__main__':
  main()


AttributeError: 'str' object has no attribute 'body'

In [36]:
import pandas as pd
import re

def extract_features(email):
  """Extracts features from an email message.

  Args:
    email: The email message as a Pandas Series.

  Returns:
    A dictionary of features.
  """

  features = {}

  # Number of words in the body.
  body_words = re.findall(r"\w+", email['Cleaned Body'])
  features["body_words"] = len(body_words)

  # Number of characters in the body.
  body_characters = len(email['Cleaned Body'])
  features["body_characters"] = body_characters

  # Number of words in the subject.
  subject_words = re.findall(r"\w+", email['Subject'])
  features["subject_words"] = len(subject_words)

  # Number of characters in the subject.
  subject_characters = len(email['Subject'])
  features["subject_characters"] = subject_characters

  # Body richness.
  features["body_richness"] = features["body_words"] / features["body_characters"]

  # Subject richness.
  features["subject_richness"] = features["subject_words"] / features["subject_characters"]

  # Number of links in the email.
  links = re.findall(r"(http|https)://\S+", email['Cleaned Body'])
  features["url_count"] = len(links)

  # Number of external links in the email.
  external_links = [link for link in links if not link.startswith("http://localhost")]
  features["url_external_count"] = len(external_links)

  # Number of domains in the email.
  domains = set([link.split("/")[2] for link in links])
  features["url_domain_count"] = len(domains)

  # Whether the body contains HTML.
  if re.search(r"<html>", email['Cleaned Body']):
    features["body_html"] = 1
  else:
    features["body_html"] = 0

  # Maximum number of periods in a URL.
  max_periods = 0
  for link in links:
    num_periods = link.count(".")
    if num_periods > max_periods:
      max_periods = num_periods
  features["url_max_periods"] = max_periods

  return features

def main():
  df = pd.read_csv('../data/combined_data/combined_data_cleaned.csv')

  # Create a new DataFrame with the features.
  new_df = pd.DataFrame()
  for index, row in df.iterrows():
    email = row['Cleaned Body']
    features = extract_features(email)
    new_df = new_df.append(features, ignore_index=True)

  print(new_df.head())

if __name__ == '__main__':
  main()


TypeError: string indices must be integers

In [37]:
import pandas as pd
import re

def extract_features(email):
  """Extracts features from an email message.

  Args:
    email: The email message as a Pandas Series.

  Returns:
    A dictionary of features.
  """

  features = {}

  # Number of words in the body.
  body_words = email['Cleaned Body'].split()
  features["body_words"] = len(body_words)

  # Number of characters in the body.
  body_characters = len(email['Cleaned Body'])
  features["body_characters"] = body_characters

  # Number of words in the subject.
  subject_words = email['Subject'].split()
  features["subject_words"] = len(subject_words)

  # Number of characters in the subject.
  subject_characters = len(email['Subject'])
  features["subject_characters"] = subject_characters

  # Body richness.
  features["body_richness"] = features["body_words"] / features["body_characters"]

  # Subject richness.
  features["subject_richness"] = features["subject_words"] / features["subject_characters"]

  # Number of links in the email.
  links = re.findall(r"(http|https)://\S+", email['Cleaned Body'])
  features["url_count"] = len(links)

  # Number of external links in the email.
  external_links = [link for link in links if not link.startswith("http://localhost")]
  features["url_external_count"] = len(external_links)

  # Number of domains in the email.
  domains = set([link.split("/")[2] for link in links])
  features["url_domain_count"] = len(domains)

  # Whether the body contains HTML.
  if re.search(r"<html>", email['Cleaned Body']):
    features["body_html"] = 1
  else:
    features["body_html"] = 0

  # Maximum number of periods in a URL.
  max_periods = 0
  for link in links:
    num_periods = link.count(".")
    if num_periods > max_periods:
      max_periods = num_periods
  features["url_max_periods"] = max_periods

  return features

def main():
  df = pd.read_csv('../data/combined_data/combined_data_cleaned.csv')

  # Create a new DataFrame with the features.
  new_df = pd.DataFrame()
  for index, row in df.iterrows():
    email = row['Cleaned Body']
    features = extract_features(email)
    new_df = new_df.append(features, ignore_index=True)

  print(new_df.head())

if __name__ == '__main__':
  main()


TypeError: string indices must be integers